In [46]:
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import janitor as ja
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [47]:
raw_airbnb=pd.read_csv("data/listings.csv")
london_pois=gpd.read_file("data/pois/gis_osm_pois_free_1.shp")
msoa_raw_boundary=gpd.read_file("data/greater_london.gpkg")

In [48]:
geometry = [Point(xy) for xy in zip(raw_airbnb['longitude'], raw_airbnb['latitude'])]
geo_airbnb = gpd.GeoDataFrame(raw_airbnb, geometry=geometry, crs="EPSG:4326")
# import Airbnb list，transform it into GeoDataFrame

In [49]:
pois_27700 = london_pois.to_crs("EPSG:27700")
geo_airbnb=geo_airbnb.to_crs("EPSG:27700")

london_pois = GeoDataFrame(london_pois, geometry=london_pois['geometry'])
geo_airbnb = GeoDataFrame(geo_airbnb, geometry=geo_airbnb['geometry'])
#reproject the coordinate system

In [50]:
#drop some unuse columns
geo_airbnb=geo_airbnb.drop(columns=['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'calculated_host_listings_count','availability_365'])

In [51]:
airbnb_intersect = gpd.sjoin(geo_airbnb, msoa_raw_boundary, how="inner", predicate="intersects")
#交汇爱彼迎数据与空间

In [52]:
airbnb_intersect

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,number_of_reviews_ltm,license,geometry,index_right,geo_code,geo_label,geo_labelw
0,Enfield,51.611370,-0.118880,Private room,40.0,7,38,2024-01-01,0.28,1,NaN,POINT (530350.368 191957.865),978,E02000312,Enfield 036,None
1,Hammersmith and Fulham,51.501550,-0.233002,Entire home/apt,131.0,5,91,2024-06-09,0.59,10,NaN,POINT (522743.872 179548.046),899,E02000376,Hammersmith and Fulham 005,None
2,Camden,51.559528,-0.144319,Entire home/apt,215.0,5,216,2024-07-09,1.35,5,NaN,POINT (528735.364 186147.794),836,E02000168,Camden 003,None
3,Richmond upon Thames,51.447770,-0.339620,Private room,NaN,2,0,NaN,NaN,0,NaN,POINT (515479.62 173394.164),338,E02000797,Richmond upon Thames 014,None
4,Lambeth,51.491476,-0.111514,Entire home/apt,150.0,30,148,2024-06-15,0.93,12,NaN,POINT (531204.63 178638.342),829,E02000619,Lambeth 002,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96177,Camden,51.547717,-0.143016,Entire home/apt,155.0,1,0,NaN,NaN,0,NaN,POINT (528859.074 184836.65),847,E02000172,Camden 007,None
96178,Southwark,51.505380,-0.091734,Entire home/apt,238.0,5,0,NaN,NaN,0,NaN,POINT (532537.407 180220.082),118,E02000808,Southwark 002,None
96179,Southwark,51.505380,-0.091734,Entire home/apt,188.0,5,0,NaN,NaN,0,NaN,POINT (532537.407 180220.082),118,E02000808,Southwark 002,None
96180,Southwark,51.505380,-0.091734,Entire home/apt,457.0,5,0,NaN,NaN,0,NaN,POINT (532537.407 180220.082),118,E02000808,Southwark 002,None


In [53]:
categories = london_pois['fclass'].unique()
categories
#查看poi类型

array(['waste_basket', 'bicycle_rental', 'tourist_info', 'pub',
       'restaurant', 'post_box', 'camera_surveillance', 'library',
       'post_office', 'telephone', 'fountain', 'artwork', 'toilet',
       'cafe', 'pitch', 'recycling', 'bar', 'cinema', 'memorial',
       'monument', 'recycling_glass', 'convenience', 'furniture_shop',
       'laundry', 'courthouse', 'school', 'sports_centre', 'attraction',
       'supermarket', 'university', 'beverages', 'fast_food',
       'fire_station', 'water_tower', 'newsagent', 'doctors',
       'department_store', 'museum', 'hotel', 'playground', 'gift_shop',
       'atm', 'veterinary', 'pharmacy', 'dentist', 'chemist', 'theatre',
       'bank', 'bicycle_shop', 'car_rental', 'bakery', 'hairdresser',
       'doityourself', 'comms_tower', 'hospital', 'car_sharing',
       'archaeological', 'town_hall', 'nursing_home', 'embassy',
       'recycling_paper', 'college', 'viewpoint', 'kindergarten',
       'recycling_clothes', 'public_building', 'arts_ce

In [54]:
filtered_pois = pois_27700[pois_27700['fclass'].isin(['hotel', 'guesthouse', 'motel', 'hostel', 'chalet'])]
#从osm过滤与住宿酒店有关的poi

In [55]:
#交汇poi和msoa
pois_intersect = gpd.sjoin(filtered_pois,msoa_raw_boundary, how="inner", predicate="intersects")

In [56]:
pois_intersect

,osm_id,code,fclass,name,geometry,index_right,geo_code,geo_label,geo_labelw
216,29372721,2401,hotel,Sundridge Park hotel and conference Ctr,POINT (541760.234 170636.789),783,E02006782,Bromley 040,None
315,31314241,2401,hotel,DoubleTree by Hilton Hotel London - Marble Arch,POINT (528007.431 181126.739),531,E02000971,Westminster 012,None
674,59931333,2401,hotel,easyHotel Victoria,POINT (529086.504 178641.128),331,E02000981,Westminster 022,None
1331,254025974,2401,hotel,Crestfield Hotel,POINT (530338.674 182875.701),341,E02000189,Camden 024,None
1332,254025975,2401,hotel,Kings Cross Inn Hotel,POINT (530301.093 182931.054),341,E02000189,Camden 024,None
...,...,...,...,...,...,...,...,...,...
89067,12139667177,2401,hotel,Spark by Hilton London Romford,POINT (551364.683 188537.227),27,E02000476,Havering 013,None
89245,12152771492,2401,hotel,Hotel Xanadu,POINT (517653.657 180568.267),363,E02000257,Ealing 020,None
89402,12167570871,2401,hotel,Travelodge,POINT (518239.169 169276.216),102,E02000606,Kingston upon Thames 009,None
89883,12199188775,2401,hotel,Diana Hotel,POINT (522143.571 183036.695),684,E02000123,Brent 031,None


In [57]:
accommodation_summerize=(
    pois_intersect
    .groupby(['geo_code','fclass'])
    .agg({
        'geo_label': 'first',
    })

    .reset_index()
)
#分类酒店类型和区域的列表

In [58]:
accommodation_summerize#分类酒店类型和区域的列表

,geo_code,fclass,geo_label
0,E02000001,guesthouse,City of London 001
1,E02000001,hostel,City of London 001
2,E02000001,hotel,City of London 001
3,E02000016,hotel,Barking and Dagenham 015
4,E02000031,hotel,Barnet 008
...,...,...,...
215,E02006802,hostel,Southwark 034
216,E02006854,hotel,Tower Hamlets 033
217,E02006928,hostel,Greenwich 035
218,E02006928,hotel,Greenwich 035


In [59]:
#汇总酒店数量并且重命名列
hotel_summarize = (
    accommodation_summerize
    .groupby(['geo_code'])
    .agg({
        'geo_label': 'first',  # 获取每个分组的 geo_label 第一条记录
        'fclass': 'count'      # 统计每组的 fclass 数量（如酒店数量）
    })
    .reset_index()
)

hotel_summarize.rename(columns={'fclass': 'count'}, inplace=True)

# 查看结果
print(hotel_summarize)

      geo_code                 geo_label  count
0    E02000001        City of London 001      3
1    E02000016  Barking and Dagenham 015      1
2    E02000031                Barnet 008      1
3    E02000051                Barnet 028      1
4    E02000057                Barnet 034      1
..         ...                       ...    ...
181  E02006801               Lambeth 036      2
182  E02006802             Southwark 034      1
183  E02006854         Tower Hamlets 033      1
184  E02006928             Greenwich 035      2
185  E02006930             Greenwich 037      1

[186 rows x 3 columns]


In [60]:
#import turnover data and clean names
retail=pd.read_csv('data/47 Retail trade, except of motor vehicles and motorcycles.csv',skiprows=7)
accommodation=pd.read_csv('data/55Accommodation.csv',skiprows=7)
food_and_beverage_service=pd.read_csv('data/56 Food and beverage service activities.csv',skiprows=7)
rental_and_leasing=pd.read_csv('data/77 Rental and leasing activities.csv',skiprows=7)
travel_agency_and_tour_operator=pd.read_csv('data/79 Travel agency, tour operator and other reservation service and related activities.csv',skiprows=7)
creative_art_entertainment=pd.read_csv('data/90 Creative, arts and entertainment activities.csv',skiprows=7)
gambling=pd.read_csv('data/92 Gambling and betting activities.csv',skiprows=7)
sports_amusement_recreation=pd.read_csv('data/93 Sports activities and amusement and recreation activities.csv',skiprows=7)

In [61]:
retail.columns

Index(['2011 super output area - middle layer', '0 to 49 (thousand)',
       '50 to 99  (thousand)', '100 to 249 (thousand)',
       '250 to 499 (thousand)', '500 to 999 (thousand)',
       '1000 to 1999 (thousand)', '2000 to 4999 (thousand)',
       '5000 to 9999 (thousand)', '10000 to 49999 (thousand)',
       '50000+ (thousand)'],
      dtype='object')

In [62]:
def clean_and_suffix(df, suffix):
    
    # clean the columns' name，remove the '(thousand)' 
    df.columns = df.columns.str.replace(r' \(thousand\)', '', regex=True)
    # add some tails
    df2 = df.add_suffix(suffix).clean_names()
    # delete the useless rows
    df2 = df2.drop(df2.index[np.r_[-9:1]])
    return df2

In [85]:
retail2 =clean_and_suffix(retail, '_retail')
accommodation2 =clean_and_suffix(accommodation, '_acmd')
food_bev_ser2 =clean_and_suffix(food_and_beverage_service, '_f&b')
ren_and_lea2=clean_and_suffix(rental_and_leasing, '__r&l')
travel2 =clean_and_suffix(travel_agency_and_tour_operator, '_travel')
art_ent2=clean_and_suffix(creative_art_entertainment, '_art&ent')
gambling2 =clean_and_suffix(gambling, '__gambling')
spo_amus_recre2 =clean_and_suffix(sports_amusement_recreation, '_sp_recrea')

In [86]:
merge_turnover1 = pd.merge(retail2, accommodation2, 
                          left_on ='2011_super_output_area_middle_layer_retail',
                          right_on ='2011_super_output_area_middle_layer_acmd', how='inner')
merge_turnover2 = pd.merge(merge_turnover1, food_bev_ser2, 
                          left_on='2011_super_output_area_middle_layer_retail',
                          right_on='2011_super_output_area_middle_layer_f&b',how='inner')

In [87]:
def merge_date(RightDf, ColumnName):
    mdf = pd.merge(merge_turnover2, RightDf, 
                          left_on ='2011_super_output_area_middle_layer_retail',
                          right_on = ColumnName, how='inner')
    return mdf

In [88]:
merge_turnover2 = merge_date(ren_and_lea2, '2011_super_output_area_middle_layer_r&l')
merge_turnover2 = merge_date(travel2, '2011_super_output_area_middle_layer_travel')
merge_turnover2 = merge_date(art_ent2, '2011_super_output_area_middle_layer_art&ent')
merge_turnover2 = merge_date(gambling2, '2011_super_output_area_middle_layer_gambling')
merge_turnover2 = merge_date(spo_amus_recre2, '2011_super_output_area_middle_layer_sp_recrea')
merge_turnover2= merge_turnover2.drop_duplicates()

In [89]:
merge_turnover2

,2011_super_output_area_middle_layer_retail,0_to_49_retail,50_to_99_retail,100_to_249_retail,250_to_499_retail,500_to_999_retail,1000_to_1999_retail,2000_to_4999_retail,5000_to_9999_retail,10000_to_49999_retail,...,0_to_49_sp_recrea,50_to_99_sp_recrea,100_to_249_sp_recrea,250_to_499_sp_recrea,500_to_999_sp_recrea,1000_to_1999_sp_recrea,2000_to_4999_sp_recrea,5000_to_9999_sp_recrea,10000_to_49999_sp_recrea,50000+_sp_recrea
0,E02002559 : Darlington 001,5,5.0,10.0,5.0,5.0,0.0,0.0,0.0,0.0,...,5,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,E02002560 : Darlington 002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E02002561 : Darlington 003,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E02002562 : Darlington 004,0,0.0,15.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,E02002563 : Darlington 005,0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7196,W02000362 : Newport 016,0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7197,W02000363 : Newport 017,0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7198,W02000364 : Newport 018,0,5.0,10.0,10.0,5.0,5.0,0.0,0.0,0.0,...,0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7199,W02000365 : Newport 019,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
merge_turnover2.columns

Index(['2011_super_output_area_middle_layer_retail', '0_to_49_retail',
       '50_to_99_retail', '100_to_249_retail', '250_to_499_retail',
       '500_to_999_retail', '1000_to_1999_retail', '2000_to_4999_retail',
       '5000_to_9999_retail', '10000_to_49999_retail', '50000+_retail',
       '2011_super_output_area_middle_layer_acmd', '0_to_49_acmd',
       '50_to_99_acmd', '100_to_249_acmd', '250_to_499_acmd',
       '500_to_999_acmd', '1000_to_1999_acmd', '2000_to_4999_acmd',
       '5000_to_9999_acmd', '10000_to_49999_acmd', '50000+_acmd',
       '2011_super_output_area_middle_layer_f&b', '0_to_49_f&b',
       '50_to_99_f&b', '100_to_249_f&b', '250_to_499_f&b', '500_to_999_f&b',
       '1000_to_1999_f&b', '2000_to_4999_f&b', '5000_to_9999_f&b',
       '10000_to_49999_f&b', '50000+_f&b',
       '2011_super_output_area_middle_layer_r&l', '0_to_49_r&l',
       '50_to_99_r&l', '100_to_249_r&l', '250_to_499_r&l', '500_to_999_r&l',
       '1000_to_1999_r&l', '2000_to_4999_r&l', '5000_to_99

In [91]:
final_merge=merge_turnover2.drop(columns=['2011_super_output_area_middle_layer_acmd',
                                 '2011_super_output_area_middle_layer_f&b',
                                 '2011_super_output_area_middle_layer_r&l',
                                 '2011_super_output_area_middle_layer_travel',
                                 '2011_super_output_area_middle_layer_art&ent',
                                 '2011_super_output_area_middle_layer_gambling',
                                 '2011_super_output_area_middle_layer_sp_recrea'])

In [92]:
final_merge[['geo_code', 'geo_label']] = final_merge['2011_super_output_area_middle_layer_retail'].str.split(':', expand=True)
final_merge['geo_code']=final_merge['geo_code'].str.strip()
final_merge.columns

Index(['2011_super_output_area_middle_layer_retail', '0_to_49_retail',
       '50_to_99_retail', '100_to_249_retail', '250_to_499_retail',
       '500_to_999_retail', '1000_to_1999_retail', '2000_to_4999_retail',
       '5000_to_9999_retail', '10000_to_49999_retail', '50000+_retail',
       '0_to_49_acmd', '50_to_99_acmd', '100_to_249_acmd', '250_to_499_acmd',
       '500_to_999_acmd', '1000_to_1999_acmd', '2000_to_4999_acmd',
       '5000_to_9999_acmd', '10000_to_49999_acmd', '50000+_acmd',
       '0_to_49_f&b', '50_to_99_f&b', '100_to_249_f&b', '250_to_499_f&b',
       '500_to_999_f&b', '1000_to_1999_f&b', '2000_to_4999_f&b',
       '5000_to_9999_f&b', '10000_to_49999_f&b', '50000+_f&b', '0_to_49_r&l',
       '50_to_99_r&l', '100_to_249_r&l', '250_to_499_r&l', '500_to_999_r&l',
       '1000_to_1999_r&l', '2000_to_4999_r&l', '5000_to_9999_r&l',
       '10000_to_49999_r&l', '50000+_r&l', '0_to_49_travel', '50_to_99_travel',
       '100_to_249_travel', '250_to_499_travel', '500_to_999_t

In [93]:
turnover_boundary=pd.merge(msoa_raw_boundary, final_merge, on='geo_code', how='left')

In [94]:
turnover_boundary

,geo_code,geo_label_x,geo_labelw,geometry,2011_super_output_area_middle_layer_retail,0_to_49_retail,50_to_99_retail,100_to_249_retail,250_to_499_retail,500_to_999_retail,...,50_to_99_sp_recrea,100_to_249_sp_recrea,250_to_499_sp_recrea,500_to_999_sp_recrea,1000_to_1999_sp_recrea,2000_to_4999_sp_recrea,5000_to_9999_sp_recrea,10000_to_49999_sp_recrea,50000+_sp_recrea,geo_label_y
0,E02000730,Newham 017,None,"MULTIPOLYGON (((540612.745 184312.782, 540611....",E02000730 : Newham 017,10,15.0,35.0,20.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Newham 017
1,E02000053,Barnet 030,None,"MULTIPOLYGON (((521244.462 190052.504, 521255....",E02000053 : Barnet 030,10,10.0,25.0,10.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Barnet 030
2,E02000243,Ealing 006,None,"MULTIPOLYGON (((513445.352 183679.664, 513446....",E02000243 : Ealing 006,5,5.0,5.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Ealing 006
3,E02000793,Richmond upon Thames 010,None,"MULTIPOLYGON (((513496.024 174051.923, 513495....",E02000793 : Richmond upon Thames 010,5,10.0,20.0,10.0,5.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Richmond upon Thames 010
4,E02000077,Bexley 013,None,"MULTIPOLYGON (((546675.313 176656.042, 546675....",E02000077 : Bexley 013,0,0.0,5.0,10.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bexley 013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,E02000312,Enfield 036,None,"MULTIPOLYGON (((529906.057 191882.472, 529908....",E02000312 : Enfield 036,5,5.0,15.0,10.0,5.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,Enfield 036
979,E02000332,Greenwich 020,None,"MULTIPOLYGON (((543496.077 177407.615, 543499....",E02000332 : Greenwich 020,0,0.0,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Greenwich 020
980,E02000721,Newham 008,None,"MULTIPOLYGON (((540364.064 185095.035, 540364....",E02000721 : Newham 008,5,20.0,30.0,15.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Newham 008
981,E02000396,Hammersmith and Fulham 025,None,"MULTIPOLYGON (((525384.665 176266.117, 525383....",E02000396 : Hammersmith and Fulham 025,10,5.0,15.0,15.0,10.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hammersmith and Fulham 025


In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/meimao76/group_research/refs/heads/wqc/data/47%20Retail%20trade%2C%20except%20of%20motor%20vehicles%20and%20motorcycles.csv')

In [76]:
test.head()

,UK Business Counts - enterprises by industry and turnover size band,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,ONS Crown Copyright Reserved [from Nomis on 15...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Analysis :,standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date :,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Industry :,"47 : Retail trade, except of motor vehicles an...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Legal Status:,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
